In [41]:
import numpy as np
from scipy.sparse import csgraph

import sys
sys.path.append('../')
from src import data
from scipy.stats import spearmanr

# https://github.com/aestrivex/bctpy
# dokumentace asi jenom pro matlab? https://sites.google.com/site/bctnet/
# from bct.algorithms import efficiency

from netneurotools import metrics # zjistit, proč tohle nefunguje přes pip 
# (mám to stažené lokálně z Githubu a to pak funguje)

# https://enigma-toolbox.readthedocs.io/en/latest/pages/05.HCP/index.html
# enigmatoolbox obsahuje chybu, lokálně jsem si ji opravila, TODO na GitHub
from enigmatoolbox.datasets import load_sc, load_fc
from nilearn import plotting

PLOT=False

!date -I

2024-01-29


In [42]:
# load data F-TRACT
response_probability = np.loadtxt('../data/external/F-TRACT/DKT/probability.txt.gz')
response_amplitude = np.loadtxt('../data/external/F-TRACT/DKT/amplitude__median.txt.gz')


parcells_list = ['ctx-lh-bankssts','ctx-lh-caudalanteriorcingulate','ctx-lh-caudalmiddlefrontal','ctx-lh-cuneus','ctx-lh-entorhinal','ctx-lh-fusiform','ctx-lh-inferiorparietal','ctx-lh-inferiortemporal','ctx-lh-isthmuscingulate','ctx-lh-lateraloccipital','ctx-lh-lateralorbitofrontal','ctx-lh-lingual','ctx-lh-medialorbitofrontal','ctx-lh-middletemporal','ctx-lh-parahippocampal','ctx-lh-paracentral','ctx-lh-parsopercularis','ctx-lh-parsorbitalis','ctx-lh-parstriangularis','ctx-lh-pericalcarine','ctx-lh-postcentral','ctx-lh-posteriorcingulate','ctx-lh-precentral','ctx-lh-precuneus','ctx-lh-rostralanteriorcingulate','ctx-lh-rostralmiddlefrontal','ctx-lh-superiorfrontal','ctx-lh-superiorparietal','ctx-lh-superiortemporal','ctx-lh-supramarginal','ctx-lh-frontalpole','ctx-lh-temporalpole','ctx-lh-transversetemporal','ctx-lh-insula','ctx-rh-bankssts','ctx-rh-caudalanteriorcingulate','ctx-rh-caudalmiddlefrontal','ctx-rh-cuneus','ctx-rh-entorhinal','ctx-rh-fusiform','ctx-rh-inferiorparietal','ctx-rh-inferiortemporal','ctx-rh-isthmuscingulate','ctx-rh-lateraloccipital','ctx-rh-lateralorbitofrontal','ctx-rh-lingual','ctx-rh-medialorbitofrontal','ctx-rh-middletemporal','ctx-rh-parahippocampal','ctx-rh-paracentral','ctx-rh-parsopercularis','ctx-rh-parsorbitalis','ctx-rh-parstriangularis','ctx-rh-pericalcarine','ctx-rh-postcentral','ctx-rh-posteriorcingulate','ctx-rh-precentral','ctx-rh-precuneus','ctx-rh-rostralanteriorcingulate','ctx-rh-rostralmiddlefrontal','ctx-rh-superiorfrontal','ctx-rh-superiorparietal','ctx-rh-superiortemporal','ctx-rh-supramarginal','ctx-rh-frontalpole','ctx-rh-temporalpole','ctx-rh-transversetemporal','ctx-rh-insula']

with open('../data/external/F-TRACT/DKT/DKT.txt', 'r') as f:
    parcell_ids =  [i for i,line in enumerate(f) if line[:-1] in parcells_list]

# select ROI
response_probability = response_probability[np.ix_(parcell_ids, parcell_ids)]
response_amplitude  = response_amplitude[np.ix_(parcell_ids, parcell_ids)]

In [43]:
# load cortico-cortical structural connectivity data
SC, sc_ctx_labels, _, _ = load_sc()
# load cortico-cortical functional connectivity data
FC, fc_ctx_labels, _, _ = load_fc()

# plot cortico-cortical connectivity matrices
if PLOT:
    sc_plot = plotting.plot_matrix(SC, figure=(9, 9), labels=sc_ctx_labels, vmax=10, vmin=0, cmap='Blues')  
    fc_plot = plotting.plot_matrix(FC, figure=(9, 9), labels=fc_ctx_labels, vmax=10, vmin=0, cmap='Reds')  

# transform SC to distance matrix
L = np.where(SC==0.0,0,1/SC)

# shortcut for correlation of X with response probability matrix
def coor_with_response(X):
    r_probability = spearmanr(X.flatten(),response_probability.flatten(),nan_policy='omit')
    r_amplitude_median = spearmanr(X.flatten(),response_amplitude.flatten(),nan_policy='omit')

    print(f"Correlation with response probabilty: {r_probability.statistic:.3f} (p={r_probability.pvalue:.3f})")
    print(f"Correlation with response amplitude: {r_amplitude_median.statistic:.3f} (p={r_amplitude_median.pvalue:.3f})")

/tmp/ipykernel_57328/3479329224.py:12: RuntimeWarning: divide by zero encountered in divide
  L = np.where(SC==0.0,0,1/SC)


In [44]:
coor_with_response(SC) 
# paper p= ~0.4

Correlation with response probabilty:         0.353 (p=0.000)
Correlation with response amplitude (mad):    -0.190 (p=0.005)
Correlation with response amplitude (median): -0.203 (p=0.002)


In [45]:
coor_with_response(FC)

Correlation with response probabilty:         0.067 (p=0.018)
Correlation with response amplitude (mad):    -0.154 (p=0.022)
Correlation with response amplitude (median): -0.165 (p=0.014)


In [46]:
# shortest path efficiency
shorest_paths,_ = metrics.distance_wei_floyd(L)
shorest_path_eff = np.divide(1,shorest_paths)

coor_with_response(shorest_path_eff)

/tmp/ipykernel_57328/1122485199.py:3: RuntimeWarning: divide by zero encountered in divide
  shorest_path_eff = np.divide(1,shorest_paths)


Correlation with response probabilty:         0.434 (p=0.000)
Correlation with response amplitude (mad):    -0.098 (p=0.191)
Correlation with response amplitude (median): -0.124 (p=0.096)


In [47]:
com = metrics.communicability_wei(SC)

coor_with_response(com)

Correlation with response probabilty:         0.333 (p=0.000)
Correlation with response amplitude (mad):    -0.093 (p=0.168)
Correlation with response amplitude (median): -0.093 (p=0.170)


In [48]:
SI = metrics.search_information(SC,L)

coor_with_response(SI)

Correlation with response probabilty:         -0.423 (p=0.000)
Correlation with response amplitude (mad):    -0.016 (p=0.828)
Correlation with response amplitude (median): 0.002 (p=0.982)


In [52]:
# https://netneurolab.github.io/netneurotools/_modules/netneurotools/metrics.html#diffusion_efficiency
diffusion = metrics.diffusion_efficiency(SC)[1] # tahle funkce určitě má dostat SC, a ne délky L

coor_with_response(diffusion)

Correlation with response probabilty:         -0.032 (p=0.262)
Correlation with response amplitude (mad):    -0.265 (p=0.000)
Correlation with response amplitude (median): -0.250 (p=0.000)


/home/katterrina/fi_muni/0_diplomka/__venv__/lib/python3.10/site-packages/netneurotools/metrics.py:780: RuntimeWarning: divide by zero encountered in divide
  E_diff = np.divide(1, mfpt)


In [50]:
# tady té funkci možná dávám špatný vstup?
_,_,nav_paths,_,_ = metrics.navigation_wu(L, SC)
nav_eff = np.divide(1,nav_paths)

coor_with_response(nav_eff)

Correlation with response probabilty:         0.114 (p=0.000)
Correlation with response amplitude (mad):    -0.135 (p=0.070)
Correlation with response amplitude (median): -0.133 (p=0.074)


/tmp/ipykernel_57328/147778707.py:3: RuntimeWarning: divide by zero encountered in divide
  nav_eff = np.divide(1,nav_paths)
